In [2]:
import numpy as np
import json

In [3]:
with open('appearances_in_doc.txt', 'r') as fin:
    oc = json.loads(fin.read())

In [9]:
def amtofWords(word):
    thecounter = 0
    for x in oc.keys():
        try:
            thecounter += oc[x][word]
        except:
            pass
    return thecounter

In [13]:
amtofWords("languid")

29

In [14]:
with open('training.txt', 'r') as data:
            lines = data.read().splitlines() 

In [15]:
lines[2999]

'it brought the top of thÃ¢ chestnut in distinct relief upon the faintly illuminated sky the motion of the upper leaves betrayed to ben the position of the prey and in an instant he swung himself up to the first bough and proceeded upward i see the here in the of one of the branches he exclaimed to us as he hurried onward look out below the terrified animal on finding his about to his place of safety speedily abandoned it and we could distinctly hear him making his way to the remote extremity of the limb as soon as he had gained this point he became visible to us all clinging like an that had grown to the slender twigs which sustained him ben followed as near as he venture with his heavy bulk and began to whip the bough up and down with a vehement motion that flung the animal about through the air like a ball on the end of a rod still however the way laid kept his hold with a desperate during this operation the dogs as if engrossed with the contemplation of the success of the experimen